In [212]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-stock-data/NFLX.csv
/kaggle/input/netflix-shows/netflix_titles.csv


In [213]:
netflix_movies = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
netflix_stocks = pd.read_csv('../input/netflix-shows/netflix_titles.csv')

In [214]:
netflix_movies = netflix_movies[netflix_movies['type']=='Movie'].dropna()
netflix_movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
6,s7,Movie,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",Egypt,"June 1, 2020",2019,TV-MA,95 min,"Horror Movies, International Movies","After an awful accident, a couple admitted to ..."


Now, that we have our DataFrame ready; we can continue to doing some investigative work with rudamentary statistics. This includes descriptive statistics, including **work**** with movies in different countries and years, and the distribution of movies over time. Later, the descriptions will be used with the 

In [215]:
# Assuming the first movie listed is where this movie was filmed the most, we will replace the lists of movies
# with the first country and use that to find the top 10 countries that have movies are currently on Netflix 

movies_with_countries = netflix_movies.copy()
new=movies_with_countries['country'].str.split(',', n=5, expand=True)
countries = [('1st',0), ('2nd',1),('3rd',2),('4th',3),('5th',4),('6th',5)]
for i in countries:
    movies_with_countries[i[0]+" country"] = new[i[1]]
new_mwc = movies_with_countries.drop(columns=['show_id','type','director','cast'])
distribution_df = new_mwc[['1st country',
                                         'title']].groupby(
    '1st country'
).count().reset_index()
distribution_df

,1st country,title
0,Argentina,45
1,Australia,49
2,Austria,5
3,Bangladesh,2
4,Belgium,5
...,...,...
67,United States,1853
68,Uruguay,7
69,Venezuela,1
70,Vietnam,5


In [216]:
import altair as alt

titles_per_country = alt.Chart(distribution_df).transform_window(
    rank='rank(title)',
    sort=[alt.SortField('title', order='descending')]
).transform_filter(
    alt.datum.rank <=10
).mark_bar(color='red').encode(
    x=alt.X('title:Q',
           axis=alt.Axis(title='Number of Movies',
                         tickCount=4,
                         labelPadding=10,
                        titlePadding=10)),
    y=alt.Y('1st country:N',
           sort=alt.EncodingSortField('title',order='descending'),
           title=None),
    color=alt.condition(
        alt.datum.rank == 1,
        alt.value('red'),
        alt.value('black')
    )
).configure_view(
    strokeOpacity=0
)
titles_per_country

alt.Chart(...)

We could say Hollywood knows how to make a good movie or should we say that one actor is taking center stage and we just can't get enough?

In [217]:
directors = movies_with_countries.copy()
directors = directors[directors['country']=='United States'][['director','title','rating','listed_in']]
directors.head()


new=movies_with_countries['country'].str.split(',', n=5, expand=True)
genre= [('1st',0), ('2nd',1)]

# does genre matter?
for i in genre:
    directors[i[0]+" genre"] = new[i[1]]
top_directors = directors[['director','title','rating','listed_in']].groupby(
    'director'
)['title'].count().reset_index()

directors['listed_in'].unique()

array(['Action & Adventure, Independent Movies, Sci-Fi & Fantasy',
       'Dramas', 'Dramas, Thrillers',
       'Independent Movies, Sci-Fi & Fantasy, Thrillers', 'Comedies',
       'Action & Adventure, Sci-Fi & Fantasy', 'Dramas, Romantic Movies',
       'Dramas, Romantic Movies, Sports Movies',
       'Dramas, Independent Movies, Sports Movies',
       'Horror Movies, Independent Movies, Thrillers',
       'Horror Movies, Thrillers', 'Horror Movies, Independent Movies',
       'Documentaries, Music & Musicals', 'Dramas, Independent Movies',
       'Comedies, Dramas', 'Dramas, LGBTQ Movies',
       'Action & Adventure, Comedies', 'Documentaries',
       'Comedies, LGBTQ Movies', 'Comedies, Dramas, Independent Movies',
       'Comedies, Dramas, Romantic Movies', 'Comedies, Romantic Movies',
       'Action & Adventure, Dramas',
       'Dramas, Independent Movies, Romantic Movies',
       'Comedies, Independent Movies, Thrillers',
       'Children & Family Movies', 'Dramas, Sci-Fi & Fant

In [218]:
# does the year matter
# netflix_movies.date_added = pd.to_datetime(netflix_data.date_added)
# movies_over_time = netflix_movies.copy().groupby('release_year').apply(lambda x: len(x)).reset_index()
# movies_over_time = movies_over_time.rename(columns={0:'count'})
netflix_movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,2016-12-23,2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,2018-12-20,2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,2017-11-16,2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,2020-01-01,2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
6,s7,Movie,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",Egypt,2020-06-01,2019,TV-MA,95 min,"Horror Movies, International Movies","After an awful accident, a couple admitted to ..."


In [219]:
# does genre matter?
# genre = []
# for i in genre:
#     directors[i[0]+" genre"] = genre[i[1]]
# top_directors = directors[['director','title','rating','listed_in']].groupby(
#     'director'
# )['title'].count().reset_index()